In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego,Rio De Janeiro - Consumo de Cimento (t)
0,2003-1,0.291137,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,199.081280,0.773489,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,8.356512,254.362
1,2003-2,0.293538,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,198.561780,0.773547,3.587441e+08,1.814108e+07,22.149228,3.474056e+08,8.350332,252.842
2,2003-3,0.295914,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,198.773336,0.773606,3.591452e+08,1.814681e+07,22.154735,3.475266e+08,8.344152,229.385
3,2003-4,0.298264,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,198.753800,0.773665,3.595464e+08,1.815255e+07,22.160243,3.476476e+08,8.337971,243.407
4,2003-5,0.300588,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,198.721455,0.773724,3.599475e+08,1.815828e+07,22.165750,3.477686e+08,8.331791,256.535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.559828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.917513,NaN,NaN,NaN,NaN,NaN,NaN,241.702
236,2022-9,0.558289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.658000,NaN,NaN,NaN,NaN,NaN,NaN,221.036
237,2022-10,0.556367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.339116,NaN,NaN,NaN,NaN,NaN,NaN,224.399
238,2022-11,0.554021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231.667333,NaN,NaN,NaN,NaN,NaN,NaN,221.319


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
0,-1.533996,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.060779,-1.723389,-1.790542,0.136347,0.121226,-1.445055,-0.790925
1,-1.486609,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.079132,-1.706702,-1.768299,0.149077,0.152115,-1.365321,-0.794037
2,-1.439709,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.071658,-1.690015,-1.746055,0.161806,0.183005,-1.285587,-0.797148
3,-1.393318,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.072349,-1.673328,-1.723811,0.174535,0.213895,-1.205853,-0.800259
4,-1.347462,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.073491,-1.656642,-1.701567,0.187264,0.244784,-1.126118,-0.803370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.438978,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.476880,0.839598,1.110414,-1.635096,-1.325228,-1.036232,1.324172
188,1.443990,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.460898,0.793634,1.098172,-1.625552,-1.326867,-1.053116,1.322796
189,1.457014,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.421271,0.747671,1.085930,-1.616009,-1.328507,-1.070000,1.321421
190,1.470138,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.402090,0.701707,1.073688,-1.606465,-1.330147,-1.086883,1.320046


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      256.494
1      214.974
2      273.646
3      238.045
4      254.860
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Rio De Janeiro - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
0,-1.533996,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.060779,-1.723389,-1.790542,0.136347,0.121226,-1.445055,-0.790925
1,-1.486609,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.079132,-1.706702,-1.768299,0.149077,0.152115,-1.365321,-0.794037
2,-1.439709,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.071658,-1.690015,-1.746055,0.161806,0.183005,-1.285587,-0.797148
3,-1.393318,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.072349,-1.673328,-1.723811,0.174535,0.213895,-1.205853,-0.800259
4,-1.347462,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.073491,-1.656642,-1.701567,0.187264,0.244784,-1.126118,-0.803370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.395833,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-1.418673,1.438456,1.098176,-1.500012,-1.660268,-1.692180,1.473338
158,1.389960,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-1.470839,1.429097,1.105247,-1.521309,-1.646368,-1.657425,1.467585
159,1.384722,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-1.532363,1.419738,1.112318,-1.542605,-1.632469,-1.622671,1.461833
160,1.379646,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-1.608317,1.410379,1.119389,-1.563902,-1.618570,-1.587917,1.456081


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      256.494
1      214.974
2      273.646
3      238.045
4      254.860
        ...   
157    175.917
158    208.629
159    181.933
160    187.947
161    181.556
Name: Rio De Janeiro - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
126,0.575814,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.995998,0.797097,0.862422,-0.029106,-0.391329,0.186886,0.731632
127,0.620950,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.926846,0.824296,0.875882,-0.083792,-0.444296,0.111052,0.766827
128,0.671632,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.836461,0.851494,0.889342,-0.138478,-0.497262,0.035217,0.802021
129,0.720432,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.767757,0.878692,0.902802,-0.193163,-0.550229,-0.040617,0.837216
130,0.766028,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.681375,0.905890,0.916261,-0.247849,-0.603195,-0.116452,0.872410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.438978,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.476880,0.839598,1.110414,-1.635096,-1.325228,-1.036232,1.324172
188,1.443990,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.460898,0.793634,1.098172,-1.625552,-1.326867,-1.053116,1.322796
189,1.457014,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.421271,0.747671,1.085930,-1.616009,-1.328507,-1.070000,1.321421
190,1.470138,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.402090,0.701707,1.073688,-1.606465,-1.330147,-1.086883,1.320046


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3801222292, 1978486284, 540494382, 2953801142, 318025729, 3993341451, 3555759444, 2443032214, 1881458932, 445128848, 141017334, 2970296261, 478620990, 3198409524, 3980397563, 1262005573, 3073224327, 2635827235, 4055507767, 996943586, 3208194345, 1000242926, 4042593055, 2224223060, 695276716, 795432374, 746356171, 2188186858, 2537597267, 2514070932, 4157612354, 4137359500, 826972300, 964247394, 248303300, 2618747496, 1419239825, 1044780594, 2718584614, 3134498934, 2343839332, 1506929395, 1696652059, 3796495804, 2277188766, 1266030275, 75357826, 1871014032, 3935201111, 1091513706]


Step: 0 ___________________________________________
val_loss: 1296.3265380859375
winner_seed: 3801222292


Step: 1 ___________________________________________
val_loss: 870.3244018554688
winner_seed: 1978486284


Step: 2 ___________________________________________
val_loss: 692.8092651367188
winner_seed: 540494382


Step: 3 ___________________________________________
val_loss: 687.242431640625
winner_seed: 2

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 373ms/step - loss: 76958.9141 - val_loss: 92023.8750
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 160867.5312 - val_loss: 43813.8281
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 21221.9297 - val_loss: 8397.5674
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 5167.0137 - val_loss: 4362.4121
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 4156.3394 - val_loss: 7117.6460
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 4990.6885 - val_loss: 4682.4419
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 4407.2666 - val_loss: 5184.6431
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 3719.1584 - val_loss: 5667.1099
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 4423.6367 - val_loss: 5717.4243
Epoch 10/10000
4/4 [===================

4/4 [==============================] - 0s 10ms/step - loss: 1405.7539 - val_loss: 2834.2053
Epoch 78/10000
4/4 [==============================] - 0s 10ms/step - loss: 2002.0962 - val_loss: 1355.9702
Epoch 79/10000
4/4 [==============================] - 0s 10ms/step - loss: 1345.0182 - val_loss: 1376.1879
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 1067.8337 - val_loss: 2023.6582
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 1326.9124 - val_loss: 1530.8372
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 940.5788 - val_loss: 1381.0863
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 1181.0916 - val_loss: 1515.7617
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 1858.7748 - val_loss: 1807.5383
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 1578.0697 - val_loss: 1587.6105
Epoch 86/10000
4/4 [==============================] -

4/4 [==============================] - 0s 10ms/step - loss: 815.5009 - val_loss: 2479.4441
Epoch 154/10000
4/4 [==============================] - 0s 10ms/step - loss: 1122.3949 - val_loss: 1488.9899
Epoch 155/10000
4/4 [==============================] - 0s 10ms/step - loss: 1150.7357 - val_loss: 1274.3810
Epoch 156/10000
4/4 [==============================] - 0s 10ms/step - loss: 925.6925 - val_loss: 1400.9142
Epoch 157/10000
4/4 [==============================] - 0s 10ms/step - loss: 746.9441 - val_loss: 1263.0875
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 702.3773 - val_loss: 1901.4427
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 1167.7925 - val_loss: 1423.5803
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 1104.1342 - val_loss: 1613.6666
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 801.0618 - val_loss: 1534.2081
Epoch 162/10000
4/4 [============================

4/4 [==============================] - 0s 10ms/step - loss: 1080.1206 - val_loss: 1191.7052
Epoch 230/10000
4/4 [==============================] - 0s 10ms/step - loss: 1815.0717 - val_loss: 2984.0225
Epoch 231/10000
4/4 [==============================] - 0s 10ms/step - loss: 1669.5150 - val_loss: 1661.2153
Epoch 232/10000
4/4 [==============================] - 0s 10ms/step - loss: 1390.3771 - val_loss: 1756.6099
Epoch 233/10000
4/4 [==============================] - 0s 10ms/step - loss: 1332.4548 - val_loss: 1508.5784
Epoch 234/10000
4/4 [==============================] - 0s 10ms/step - loss: 1036.6464 - val_loss: 1267.4785
Epoch 235/10000
4/4 [==============================] - 0s 10ms/step - loss: 1046.8790 - val_loss: 2122.8081
Epoch 236/10000
4/4 [==============================] - 0s 10ms/step - loss: 1525.4991 - val_loss: 1662.9552
Epoch 237/10000
4/4 [==============================] - 0s 10ms/step - loss: 1444.8817 - val_loss: 1839.1102
Epoch 238/10000
4/4 [=======================

4/4 [==============================] - 0s 10ms/step - loss: 1080.2391 - val_loss: 1857.8038
Epoch 306/10000
4/4 [==============================] - 0s 10ms/step - loss: 866.9958 - val_loss: 1348.1530
Epoch 307/10000
4/4 [==============================] - 0s 10ms/step - loss: 724.9031 - val_loss: 1781.5415
Epoch 308/10000
4/4 [==============================] - 0s 10ms/step - loss: 739.5910 - val_loss: 1358.7050
Epoch 309/10000
4/4 [==============================] - 0s 10ms/step - loss: 659.0062 - val_loss: 1354.4001
Epoch 310/10000
4/4 [==============================] - 0s 10ms/step - loss: 657.8566 - val_loss: 1334.3424
Epoch 311/10000
4/4 [==============================] - 0s 10ms/step - loss: 930.6234 - val_loss: 1413.4847
Epoch 312/10000
4/4 [==============================] - 0s 10ms/step - loss: 761.5222 - val_loss: 1551.6788
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 765.5579 - val_loss: 1439.4402
Epoch 314/10000
4/4 [==============================]

...

Epoch 1827/10000
4/4 [==============================] - 0s 10ms/step - loss: 133.5782 - val_loss: 610.7246
Epoch 1828/10000
4/4 [==============================] - 0s 10ms/step - loss: 104.9055 - val_loss: 672.4622
Epoch 1829/10000
4/4 [==============================] - 0s 10ms/step - loss: 110.8185 - val_loss: 634.7070
Epoch 1830/10000
4/4 [==============================] - 0s 10ms/step - loss: 112.3253 - val_loss: 625.6866
Epoch 1831/10000
4/4 [==============================] - 0s 10ms/step - loss: 68.5930 - val_loss: 689.1654
Epoch 1832/10000
4/4 [==============================] - 0s 10ms/step - loss: 125.1317 - val_loss: 694.2050
Epoch 1833/10000
4/4 [==============================] - 0s 10ms/step - loss: 116.6367 - val_loss: 590.5981
Epoch 1834/10000
4/4 [==============================] - 0s 10ms/step - loss: 110.4400 - val_loss: 831.6858
Epoch 1835/10000
4/4 [==============================] - 0s 10ms/step - loss: 125.9644 - val_loss: 769.8196
Epoch 1836/10000
4/4 [================

4/4 [==============================] - 0s 10ms/step - loss: 184.1295 - val_loss: 790.9198
Epoch 1904/10000
4/4 [==============================] - 0s 10ms/step - loss: 190.6846 - val_loss: 837.2625
Epoch 1905/10000
4/4 [==============================] - 0s 10ms/step - loss: 168.2956 - val_loss: 579.7286
Epoch 1906/10000
4/4 [==============================] - 0s 10ms/step - loss: 131.1172 - val_loss: 635.6587
Epoch 1907/10000
4/4 [==============================] - 0s 10ms/step - loss: 314.9456 - val_loss: 628.6694
Epoch 1908/10000
4/4 [==============================] - 0s 10ms/step - loss: 450.3391 - val_loss: 556.8241
Epoch 1909/10000
4/4 [==============================] - 0s 10ms/step - loss: 274.6181 - val_loss: 586.5579
Epoch 1910/10000
4/4 [==============================] - 0s 10ms/step - loss: 196.7907 - val_loss: 780.1599
Epoch 1911/10000
4/4 [==============================] - 0s 10ms/step - loss: 217.5516 - val_loss: 684.0037
Epoch 1912/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 127.9890 - val_loss: 956.7984
Epoch 1980/10000
4/4 [==============================] - 0s 10ms/step - loss: 393.3333 - val_loss: 678.2317
Epoch 1981/10000
4/4 [==============================] - 0s 10ms/step - loss: 220.1210 - val_loss: 726.1735
Epoch 1982/10000
4/4 [==============================] - 0s 10ms/step - loss: 230.1335 - val_loss: 667.1415
Epoch 1983/10000
4/4 [==============================] - 0s 10ms/step - loss: 230.6492 - val_loss: 618.7338
Epoch 1984/10000
4/4 [==============================] - 0s 10ms/step - loss: 173.7837 - val_loss: 969.9868
Epoch 1985/10000
4/4 [==============================] - 0s 10ms/step - loss: 225.1919 - val_loss: 859.9523
Epoch 1986/10000
4/4 [==============================] - 0s 10ms/step - loss: 231.5571 - val_loss: 766.1409
Epoch 1987/10000
4/4 [==============================] - 0s 10ms/step - loss: 177.4507 - val_loss: 663.0342
Epoch 1988/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 99.7546 - val_loss: 881.2200
Epoch 2056/10000
4/4 [==============================] - 0s 10ms/step - loss: 145.7044 - val_loss: 915.4275
Epoch 2057/10000
4/4 [==============================] - 0s 9ms/step - loss: 238.2563 - val_loss: 679.6829
Epoch 2058/10000
4/4 [==============================] - 0s 10ms/step - loss: 201.7199 - val_loss: 782.3720
Epoch 2059/10000
4/4 [==============================] - 0s 10ms/step - loss: 137.5204 - val_loss: 722.8190
Epoch 2060/10000
4/4 [==============================] - 0s 10ms/step - loss: 100.9594 - val_loss: 746.3463
Epoch 2061/10000
4/4 [==============================] - 0s 10ms/step - loss: 112.7819 - val_loss: 792.4949
Epoch 2062/10000
4/4 [==============================] - 0s 10ms/step - loss: 151.3361 - val_loss: 635.2540
Epoch 2063/10000
4/4 [==============================] - 0s 10ms/step - loss: 147.7794 - val_loss: 847.3027
Epoch 2064/10000
4/4 [==============================] - 

Epoch 2132/10000
4/4 [==============================] - 0s 10ms/step - loss: 81.0123 - val_loss: 770.3357
Epoch 2133/10000
4/4 [==============================] - 0s 10ms/step - loss: 68.1956 - val_loss: 842.1064
Epoch 2134/10000
4/4 [==============================] - 0s 10ms/step - loss: 71.8098 - val_loss: 811.2955
Epoch 2135/10000
4/4 [==============================] - 0s 10ms/step - loss: 109.3200 - val_loss: 918.0869
Epoch 2136/10000
4/4 [==============================] - 0s 10ms/step - loss: 135.6684 - val_loss: 730.3292
Epoch 2137/10000
4/4 [==============================] - 0s 10ms/step - loss: 118.7019 - val_loss: 853.4911
Epoch 2138/10000
4/4 [==============================] - 0s 10ms/step - loss: 172.4346 - val_loss: 681.5952
Epoch 2139/10000
4/4 [==============================] - 0s 10ms/step - loss: 144.4507 - val_loss: 772.0470
Epoch 2140/10000
4/4 [==============================] - 0s 10ms/step - loss: 114.6919 - val_loss: 740.2614
Epoch 2141/10000
4/4 [==================

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,202.894135,202.94574,203.301086,203.221405,203.683136,210.034653,191.838638,189.335815,188.721436,188.519867,189.082413,193.948288,193.928207,194.301453,193.766373,194.438263,194.544525,194.454147,194.046585,193.222015,193.888779,192.79837,192.595062,192.527252,192.321701,190.659698,190.482574,190.893387,193.492386,195.799561
Target,185.846,200.671,207.913,188.107,193.661,175.783,185.078,146.761,173.121,169.841,142.706,242.422,226.205,229.696,232.624,234.262,217.727,190.769,232.609,195.364,205.69,231.354,236.711,215.36,258.015,244.87,225.41,252.849,226.618,190.993
Error,17.048141,2.274734,4.611908,15.11441,10.022141,34.251648,6.760635,42.574814,15.600433,18.678864,46.376419,48.473709,32.276794,35.394547,38.85762,39.82373,23.18248,3.68515,38.562408,2.141983,11.801224,38.555634,44.115936,22.832748,65.693314,54.210297,34.927429,61.955612,33.12561,4.806564


In [38]:
display(mae)
display(mape)

28.257896

0.13346478

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[2367.5266]] - Target[2211.91]| =  Error: [[155.6167]]; MAPE:[[0.07035399]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[2324.5112]] - Target[2648.3710000000005]| =  Error: [[323.85986]]; MAPE:[[0.12228644]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[1153.6493]] - Target[1398.7549999999999]| =  Error: [[245.10571]]; MAPE:[[0.17523134]]


[array([[155.6167]], dtype=float32),
 array([[323.85986]], dtype=float32),
 array([[245.10571]], dtype=float32)]

241.52742

0.12262392